In [1]:
# !pip install pandas
# !pip install openpyxl

In [2]:
import pandas as pd
import glob

## 최신 법정동 코드 DF

- 법정동명이 '서울특별시 OO구'인 것만 남기고, 'OO구'로 변환
- 딕셔너리로 생성 : {'법정동명':법정동코드}

In [3]:
new_code_df = pd.read_excel('../dedev_2nd_project/법정동코드 조회자료.xlsx')
new_code_df.head()

,법정동코드,법정동명
0,1100000000,서울특별시
1,1111000000,서울특별시 종로구
2,1111010100,서울특별시 종로구 청운동
3,1111010200,서울특별시 종로구 신교동
4,1111010300,서울특별시 종로구 궁정동


In [4]:
filtered_gucode_df = new_code_df[new_code_df['법정동명'].str.endswith('구')]
new_code_dic = {row['법정동명'].split()[-1]: row['법정동코드'] for _, row in filtered_gucode_df.iterrows()}
print(new_code_dic)
print('서울특별시의 자치구 수 :', len(new_code_dic))

{'종로구': 1111000000, '중구': 1114000000, '용산구': 1117000000, '성동구': 1120000000, '광진구': 1121500000, '동대문구': 1123000000, '중랑구': 1126000000, '성북구': 1129000000, '강북구': 1130500000, '도봉구': 1132000000, '노원구': 1135000000, '은평구': 1138000000, '서대문구': 1141000000, '마포구': 1144000000, '양천구': 1147000000, '강서구': 1150000000, '구로구': 1153000000, '금천구': 1154500000, '영등포구': 1156000000, '동작구': 1159000000, '관악구': 1162000000, '서초구': 1165000000, '강남구': 1168000000, '송파구': 1171000000, '강동구': 1174000000}
서울특별시의 자치구 수 : 25


## 행정동단위 29개 통신정보 전처리

### 데이터 읽어오기

In [6]:
file_list = glob.glob('../dedev_2nd_project/23_commuinfo_lifedata/*.xlsx')
all_commuinfo_life_df = []

for m, file_path in enumerate(file_list):
    df = pd.read_excel(file_path)
    if m + 1 >= 10:
        df['month'] = str(m+1)
    else:
        df['month'] = '0' + str(m+1)
    all_commuinfo_life_df.append(df)

commuinfo_life_df = pd.concat(all_commuinfo_life_df, ignore_index=True)

### 1.컬럼 삭제
    - 4분위수, 미추정 컬럼들 제외

In [7]:
select_col = [
    'month', '행정동코드', '자치구', '성별', '연령대', '총인구수', '1인가구수', '야간상주지 변경횟수 평균', '주간상주지 변경횟수 평균',
    '평균 출근 소요시간 평균', '평균 근무시간 평균', '소액결재 사용횟수 평균', '소액결재 사용금액 평균', '최근 3개월 내 요금 연체 비율',
    '카카오톡 비사용 인구수', 'SNS 사용횟수', '평균 통화량', '평균 문자량', '평균 통화대상자 수', '평균 문자대상자 수', '데이터 사용량',
    '평일 총 이동 횟수', '휴일 총 이동 횟수 평균', '집 추정 위치 평일 총 체류시간', '집 추정 위치 휴일 총 체류시간', '평일 총 이동 거리 합계',
    '휴일 총 이동 거리 합계', '지하철이동일수 합계', '게임 서비스 사용일수', '금융 서비스 사용일수', '쇼핑 서비스 사용일수', '동영상/방송 서비스 사용일수',
    '유튜브 사용일수', '넷플릭스 사용일수', '배달 서비스 사용일수', '배달_브랜드 서비스 사용일수', '배달_식재료 서비스 사용일수']
del_commuinfo_life_df = commuinfo_life_df[select_col]
del_commuinfo_life_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91584 entries, 0 to 91583
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   month              91584 non-null  object 
 1   행정동코드              91584 non-null  int64  
 2   자치구                91584 non-null  object 
 3   성별                 91584 non-null  int64  
 4   연령대                91584 non-null  int64  
 5   총인구수               91584 non-null  float64
 6   1인가구수              91584 non-null  float64
 7   야간상주지 변경횟수 평균      91584 non-null  float64
 8   주간상주지 변경횟수 평균      91584 non-null  float64
 9   평균 출근 소요시간 평균      91584 non-null  float64
 10  평균 근무시간 평균         91584 non-null  float64
 11  소액결재 사용횟수 평균       91584 non-null  float64
 12  소액결재 사용금액 평균       91584 non-null  float64
 13  최근 3개월 내 요금 연체 비율  91584 non-null  float64
 14  카카오톡 비사용 인구수       91584 non-null  float64
 15  SNS 사용횟수           91584 non-null  float64
 16  평균 통화량             915

### 2. 법정동코드 최신 코드로 변경
    - 코드값 변경
    - 자치구 컬럼 삭제

In [8]:
for key, val in new_code_dic.items():
    del_commuinfo_life_df.loc[del_commuinfo_life_df['자치구'] == key, '행정동코드'] = val

del_gu_df = del_commuinfo_life_df.drop('자치구', axis=1)
del_gu_df.head()

,month,행정동코드,성별,연령대,총인구수,1인가구수,야간상주지 변경횟수 평균,주간상주지 변경횟수 평균,평균 출근 소요시간 평균,평균 근무시간 평균,...,지하철이동일수 합계,게임 서비스 사용일수,금융 서비스 사용일수,쇼핑 서비스 사용일수,동영상/방송 서비스 사용일수,유튜브 사용일수,넷플릭스 사용일수,배달 서비스 사용일수,배달_브랜드 서비스 사용일수,배달_식재료 서비스 사용일수
0,01,1111000000,1,20,220.00,68.91,2.57,4.06,89.09,536.34,...,8.38,74.16,96.40,108.96,39.24,0.24,-0.18,14.34,2.71,1.67
1,01,1111000000,1,25,321.00,111.76,2.30,3.23,52.00,580.46,...,10.71,71.15,137.81,157.28,46.97,0.22,-0.15,18.88,2.82,1.67
2,01,1111000000,1,30,350.23,112.44,2.02,2.96,43.93,522.67,...,9.93,61.57,149.82,178.59,47.49,0.13,-0.18,20.59,2.85,1.67
3,01,1111000000,1,35,353.00,111.49,1.87,2.56,62.00,556.96,...,10.21,57.91,147.34,180.28,46.21,0.05,-0.18,18.32,3.46,1.67
4,01,1111000000,1,40,419.00,77.91,1.66,2.47,68.21,546.73,...,9.31,54.41,143.42,185.52,44.21,-0.02,-0.23,17.76,2.50,1.67


### 3. groupby
    - 행정동코드, 성별, 연령대, month

In [9]:
group_commuinfo_life_df = del_gu_df.groupby(['month', '행정동코드', '성별', '연령대']).sum().reset_index()
group_commuinfo_life_df.head()

,month,행정동코드,성별,연령대,총인구수,1인가구수,야간상주지 변경횟수 평균,주간상주지 변경횟수 평균,평균 출근 소요시간 평균,평균 근무시간 평균,...,지하철이동일수 합계,게임 서비스 사용일수,금융 서비스 사용일수,쇼핑 서비스 사용일수,동영상/방송 서비스 사용일수,유튜브 사용일수,넷플릭스 사용일수,배달 서비스 사용일수,배달_브랜드 서비스 사용일수,배달_식재료 서비스 사용일수
0,01,1111000000,1,20,6856.00,2658.17,43.75,64.83,1356.55,8639.58,...,139.37,1420.85,1841.32,2203.24,706.08,4.94,-2.37,301.73,44.88,13.36
1,01,1111000000,1,25,7420.00,2836.25,36.10,59.77,1072.67,9106.41,...,164.20,1238.30,2128.61,2674.21,776.21,4.32,-3.34,332.48,47.87,20.04
2,01,1111000000,1,30,6494.23,2229.01,33.15,48.07,1131.03,9437.18,...,173.32,1172.06,2431.92,3050.77,807.22,3.03,-3.16,362.80,46.35,21.71
3,01,1111000000,1,35,5519.00,1679.12,31.22,44.60,1116.45,9379.50,...,169.79,1132.41,2453.31,3243.73,785.60,1.50,-3.63,322.81,52.30,18.37
4,01,1111000000,1,40,6464.00,1193.36,29.60,43.42,1158.04,9445.75,...,165.83,1083.21,2343.34,3165.11,730.58,-0.05,-4.96,288.59,48.57,23.38


### 4. 성별
    - 남자(1) -> 0
    - 여자(2) -> 1

In [10]:
group_commuinfo_life_df['성별'] = group_commuinfo_life_df['성별'].replace({1:0, 2:1})
group_commuinfo_life_df.head()

,month,행정동코드,성별,연령대,총인구수,1인가구수,야간상주지 변경횟수 평균,주간상주지 변경횟수 평균,평균 출근 소요시간 평균,평균 근무시간 평균,...,지하철이동일수 합계,게임 서비스 사용일수,금융 서비스 사용일수,쇼핑 서비스 사용일수,동영상/방송 서비스 사용일수,유튜브 사용일수,넷플릭스 사용일수,배달 서비스 사용일수,배달_브랜드 서비스 사용일수,배달_식재료 서비스 사용일수
0,01,1111000000,0,20,6856.00,2658.17,43.75,64.83,1356.55,8639.58,...,139.37,1420.85,1841.32,2203.24,706.08,4.94,-2.37,301.73,44.88,13.36
1,01,1111000000,0,25,7420.00,2836.25,36.10,59.77,1072.67,9106.41,...,164.20,1238.30,2128.61,2674.21,776.21,4.32,-3.34,332.48,47.87,20.04
2,01,1111000000,0,30,6494.23,2229.01,33.15,48.07,1131.03,9437.18,...,173.32,1172.06,2431.92,3050.77,807.22,3.03,-3.16,362.80,46.35,21.71
3,01,1111000000,0,35,5519.00,1679.12,31.22,44.60,1116.45,9379.50,...,169.79,1132.41,2453.31,3243.73,785.60,1.50,-3.63,322.81,52.30,18.37
4,01,1111000000,0,40,6464.00,1193.36,29.60,43.42,1158.04,9445.75,...,165.83,1083.21,2343.34,3165.11,730.58,-0.05,-4.96,288.59,48.57,23.38


### 5. 연령대 : 20~60대(10대 간격)
    - 연령대 컬럼의 값이 일의자리가 5인 것을 0으로 변경
    - groupby : month, 행정동코드, 성별, 연령대가 같은 것을 Sum
    - 70대 삭제

In [11]:
group_commuinfo_life_df['연령대'] = group_commuinfo_life_df['연령대'] // 10 * 10
age_commuinfo_life_df = group_commuinfo_life_df.groupby(['month', '행정동코드', '성별', '연령대']).sum().reset_index()
age_commuinfo_life_df.head()

,month,행정동코드,성별,연령대,총인구수,1인가구수,야간상주지 변경횟수 평균,주간상주지 변경횟수 평균,평균 출근 소요시간 평균,평균 근무시간 평균,...,지하철이동일수 합계,게임 서비스 사용일수,금융 서비스 사용일수,쇼핑 서비스 사용일수,동영상/방송 서비스 사용일수,유튜브 사용일수,넷플릭스 사용일수,배달 서비스 사용일수,배달_브랜드 서비스 사용일수,배달_식재료 서비스 사용일수
0,01,1111000000,0,20,14276.00,5494.42,79.85,124.60,2429.22,17745.99,...,303.57,2659.15,3969.93,4877.45,1482.29,9.26,-5.71,634.21,92.75,33.40
1,01,1111000000,0,30,12013.23,3908.13,64.37,92.67,2247.48,18816.68,...,343.11,2304.47,4885.23,6294.50,1592.82,4.53,-6.79,685.61,98.65,40.08
2,01,1111000000,0,40,13015.00,2401.71,59.15,84.36,2301.50,18792.95,...,336.37,2202.84,4524.21,6172.62,1429.70,-0.34,-10.74,546.72,100.54,41.75
3,01,1111000000,0,50,12905.00,2017.66,54.93,80.42,2283.14,18366.09,...,316.83,1769.49,3192.13,4367.28,1104.62,-2.69,-15.98,413.27,97.63,28.39
4,01,1111000000,0,60,7887.00,1623.03,52.89,80.05,2282.20,17667.43,...,291.71,1460.33,1878.99,2680.18,801.46,-4.64,-20.06,309.44,50.02,23.38


In [12]:
del_age_commuinfo_life_df = age_commuinfo_life_df[age_commuinfo_life_df['연령대'] != 70]
del_age_commuinfo_life_df.head(10)

,month,행정동코드,성별,연령대,총인구수,1인가구수,야간상주지 변경횟수 평균,주간상주지 변경횟수 평균,평균 출근 소요시간 평균,평균 근무시간 평균,...,지하철이동일수 합계,게임 서비스 사용일수,금융 서비스 사용일수,쇼핑 서비스 사용일수,동영상/방송 서비스 사용일수,유튜브 사용일수,넷플릭스 사용일수,배달 서비스 사용일수,배달_브랜드 서비스 사용일수,배달_식재료 서비스 사용일수
0,01,1111000000,0,20,14276.00,5494.42,79.85,124.60,2429.22,17745.99,...,303.57,2659.15,3969.93,4877.45,1482.29,9.26,-5.71,634.21,92.75,33.40
1,01,1111000000,0,30,12013.23,3908.13,64.37,92.67,2247.48,18816.68,...,343.11,2304.47,4885.23,6294.50,1592.82,4.53,-6.79,685.61,98.65,40.08
2,01,1111000000,0,40,13015.00,2401.71,59.15,84.36,2301.50,18792.95,...,336.37,2202.84,4524.21,6172.62,1429.70,-0.34,-10.74,546.72,100.54,41.75
3,01,1111000000,0,50,12905.00,2017.66,54.93,80.42,2283.14,18366.09,...,316.83,1769.49,3192.13,4367.28,1104.62,-2.69,-15.98,413.27,97.63,28.39
4,01,1111000000,0,60,7887.00,1623.03,52.89,80.05,2282.20,17667.43,...,291.71,1460.33,1878.99,2680.18,801.46,-4.64,-20.06,309.44,50.02,23.38
6,01,1111000000,1,20,14294.87,5705.67,68.59,105.67,2066.53,17957.92,...,317.50,1416.27,3732.05,5382.02,1217.09,4.33,-4.96,585.70,78.98,53.44
7,01,1111000000,1,30,11260.00,3387.65,59.84,92.63,1897.11,17952.93,...,289.37,1519.54,4527.43,7217.58,1409.28,0.81,-6.84,612.59,95.60,53.44
8,01,1111000000,1,40,13385.00,2280.54,54.23,83.68,1995.78,17511.84,...,256.90,1601.41,4086.46,7088.52,1271.17,-2.26,-10.62,528.84,93.77,50.10
9,01,1111000000,1,50,13299.26,1980.05,52.57,83.00,2055.56,17710.04,...,245.04,1433.50,2871.78,4988.74,860.23,-4.86,-16.00,373.22,87.29,43.42
10,01,1111000000,1,60,8754.00,1798.94,52.38,85.23,2095.21,16457.56,...,219.77,1341.97,1576.56,2849.90,613.11,-6.40,-20.06,340.15,25.86,31.73


### 6. 총인구수, 1인가구수
    - 소수부분 버림

In [13]:
del_age_commuinfo_life_df['총인구수'] = del_age_commuinfo_life_df['총인구수'].astype(int)
del_age_commuinfo_life_df['1인가구수'] = del_age_commuinfo_life_df['1인가구수'].astype(int)
final_commuinfo_life_df = del_age_commuinfo_life_df
final_commuinfo_life_df.head()

/tmp/ipykernel_50060/546666478.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del_age_commuinfo_life_df['총인구수'] = del_age_commuinfo_life_df['총인구수'].astype(int)
/tmp/ipykernel_50060/546666478.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del_age_commuinfo_life_df['1인가구수'] = del_age_commuinfo_life_df['1인가구수'].astype(int)


,month,행정동코드,성별,연령대,총인구수,1인가구수,야간상주지 변경횟수 평균,주간상주지 변경횟수 평균,평균 출근 소요시간 평균,평균 근무시간 평균,...,지하철이동일수 합계,게임 서비스 사용일수,금융 서비스 사용일수,쇼핑 서비스 사용일수,동영상/방송 서비스 사용일수,유튜브 사용일수,넷플릭스 사용일수,배달 서비스 사용일수,배달_브랜드 서비스 사용일수,배달_식재료 서비스 사용일수
0,01,1111000000,0,20,14276,5494,79.85,124.60,2429.22,17745.99,...,303.57,2659.15,3969.93,4877.45,1482.29,9.26,-5.71,634.21,92.75,33.40
1,01,1111000000,0,30,12013,3908,64.37,92.67,2247.48,18816.68,...,343.11,2304.47,4885.23,6294.50,1592.82,4.53,-6.79,685.61,98.65,40.08
2,01,1111000000,0,40,13015,2401,59.15,84.36,2301.50,18792.95,...,336.37,2202.84,4524.21,6172.62,1429.70,-0.34,-10.74,546.72,100.54,41.75
3,01,1111000000,0,50,12905,2017,54.93,80.42,2283.14,18366.09,...,316.83,1769.49,3192.13,4367.28,1104.62,-2.69,-15.98,413.27,97.63,28.39
4,01,1111000000,0,60,7887,1623,52.89,80.05,2282.20,17667.43,...,291.71,1460.33,1878.99,2680.18,801.46,-4.64,-20.06,309.44,50.02,23.38


### 7. 총인구수 컬럼 변경
    - interest_life와 동일하게 총인구로 컬럼명 변경

In [14]:
final_commuinfo_life_df = final_commuinfo_life_df.rename(columns={'총인구수':'총인구'})
final_commuinfo_life_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2250 entries, 0 to 2698
Data columns (total 36 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   month              2250 non-null   object 
 1   행정동코드              2250 non-null   int64  
 2   성별                 2250 non-null   int64  
 3   연령대                2250 non-null   int64  
 4   총인구                2250 non-null   int64  
 5   1인가구수              2250 non-null   int64  
 6   야간상주지 변경횟수 평균      2250 non-null   float64
 7   주간상주지 변경횟수 평균      2250 non-null   float64
 8   평균 출근 소요시간 평균      2250 non-null   float64
 9   평균 근무시간 평균         2250 non-null   float64
 10  소액결재 사용횟수 평균       2250 non-null   float64
 11  소액결재 사용금액 평균       2250 non-null   float64
 12  최근 3개월 내 요금 연체 비율  2250 non-null   float64
 13  카카오톡 비사용 인구수       2250 non-null   float64
 14  SNS 사용횟수           2250 non-null   float64
 15  평균 통화량             2250 non-null   float64
 16  평균 문자량             2250 non-n

### CSV 저장

In [ ]:
final_commuinfo_life_df.to_csv('pre_commuinfo_life.csv', index=False)